При помощи этой конструкции экземплярам класса можно задать только определенные атрибуты
Посмотрим как это работает на примере:

In [ ]:
class Point:

    def __init__(self, x, y):
        self.x = x
        self.y = y

p = Point(12, 14)
p.new = 12
print(dir(p))

Как мы видим мы можем создавать новые атрибуты к экземпляру нашего классе вне нашего класса. И мы хотим, чтобы такого не происходило.\
Мы хотим, чтобы четко фиксированное количество атрибутов было у экземпляра нашего класса.

In [ ]:
class Point:

    __slots__ = ('x', 'y')

    def __init__(self, x, y):
        self.x = x
        self.y = y


p = Point(12, 14)
p.new = 12
# AttributeError

В магическом атрибуте \_\_slots\_\_ нашего класса Point мы указали что у экземпляров этого класса должны быть только атрибуты x и y.\
\_\_slots\_\_ должен быть неизменяемым итерируемым объектом. Он должен содержать имена атрибутов, которые будут присутствовать в экземпляре нашего класса\
\
Slots нужен для того, чтобы создать ограничение для количества атрибутов.\
Классы, которые написаны с использованием slots занимают меньше памяти\
При использовании slots требуется меньше времени на стандартные операции при работе с классами,\
а именно: добавление атрибута, удаление атрибута, изменение значения атрибута, обращение к атрибуту\
\
Чтобы убедиться в том, что действительно меньше памяти расходуется, можно запустить следующий код:

In [ ]:
class Point:

    def __init__(self, x, y):
        self.x = x
        self.y = y


class PointSlots:

    __slots__ = ('x', 'y')

    def __init__(self, x, y):
        self.x = x
        self.y = y


p = Point(12, 14)
ps = PointSlots(12, 14)
print(p.__sizeof__(), p.__dict__.__sizeof__())
print(ps.__sizeof__())

Чтобы убедиться в том, что действительно уходит меньше времени на стандартные операции, можно запустить следующий код:

In [ ]:
from timeit import timeit

class Point:

    def __init__(self, x, y):
        self.x = x
        self.y = y


class PointSlots:

    __slots__ = ('x', 'y')

    def __init__(self, x, y):
        self.x = x
        self.y = y

def make_c1():
    s = Point(3, 4)
    s.x = 100
    s.x
    del s.x


def make_c2():
    s = PointSlots(3, 4)
    s.x = 100
    s.x
    del s.x


print(timeit(make_c1))
print(timeit(make_c2))

В нём наши два класса и две функции, одна из них делает стандратные операции с экземпляром класса Point, а другая с экземпляром класса PointSlots и также используем штуку, которая позволяет измерить время выполнения функции.\
Таким образом убеждаемся в том, что с использованием slots достигается меньшее использование времени на стандратные операции над экземплярами класса\
\
Будем разбираться с slots дальше\
\
Создадим класс прямоугольник и в нём сделаем вычисляемые свойства по нахождению периметра и площади

In [ ]:
class Rectangle:

    __slots__ = ('__width', 'height')

    def __init__(self, a, b):
        self.width = a
        self.height = b


    @property
    def perimetr(self): #Вычисляемое свойство
        return (self.height + self.width) * 2


    @property
    def area(self, value): # вычисляемое свойство
        return self.height * self.width


b = Rectangle(4, 5)
print(b.area)
print(b.perimetr)

Как мы видим в slots не указаны в качестве атрибутов perimetr и area, но при этом у нас всё работает\
\
Ну раз мы уже умные, значит мы можем создать собственные геттеры и сеттеры через property

In [ ]:
class Rectangle:

    __slots__ = ('__width', 'height')

    def __init__(self, a, b):
        self.width = a
        self.height = b

    @property
    def width(self):
        return self.__width

    @width.setter
    def width(self, value):
        self.__width = value

При инициализации экземпляра класса Rectangle будет происходить следующее:\
self.width = a повлечет за собой вызов сеттера для width и тем самым будет установлен защищенный атрибут __width\
\
Хорошо, теперь мы умеем работать с property в тех классах, где есть \_\_slots\_\_\
А что же там с наследованием ?

In [ ]:
class Rectangle:

    __slots__ = ('__width', 'height')

    def __init__(self, a, b):
        self.width = a
        self.height = b

    @property
    def width(self):
        return self.__width

    @width.setter
    def width(self, value):
        self.__width = value

class Square(Rectangle):
    
    __slots__ = ('color')
    
    def __init__(self, a, b, color):
        super().__init__(a, b)
        self.color = color

d = Square(2, 2, 'red')
print(d.width)
print(d.height)
print(d.color)
d.qwerty = 123 # AttributeError

Таким образом если написать slots в дочернем классе, у родителя которого присутствует slots, то произойдет расширение уже имеющегося slots. Мы можем пользоваться атрибутами класса родителя в экземплярах классов потомка\
\
Но если мы просто отнаследуемся от класса, в котором slots имеется, то никаких ограничений на атрибуты класса потомка накладываться не будет:

In [ ]:
class Rectangle:

    __slots__ = ('__width', 'height')

    def __init__(self, a, b):
        self.width = a
        self.height = b

    @property
    def width(self):
        return self.__width

    @width.setter
    def width(self, value):
        self.__width = value


class Square(Rectangle):
    pass

d = Square(3, 3)
d.qwerty = 1234
print(d.qwerty)